<a href="https://colab.research.google.com/github/TD008/KNN-model/blob/main/KNN_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [21]:
# Importing data
f = open('mnist_kaggle.pkl', 'rb')
dictionary = pickle.load(f)
f.close()

X_train = dictionary['training_images']
y_train = dictionary['training_labels']
X_test = dictionary['test_images']
y_test = dictionary['test_labels']
num_labels = 10
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

UnpicklingError: pickle data was truncated

## 2.B Make KNN faster!

In [14]:
# Applying the avg pooling operation
# Original size of data = 28x28
data = torch.rand(1,1,28,28)
print(data.size())
pool2 = torch.nn.AvgPool2d(2, stride=2)
pool4 = torch.nn.AvgPool2d(4)

new_data = pool2(data)



torch.Size([1, 1, 28, 28])


## 2.C Make Centroid Method & Logistic Regression faster!

In [ ]:
def centroid_computation(X, y, num_centroids = 10):
    centroids = []
    for c in range(num_centroids):
        centroid = np.mean(X[y == c, :], axis = 0)
        centroids.append(centroid)
    return centroids

centroids = centroid_computation(X_train, y_train)

def save_img(img, label, file_name, num_rows = 28, num_cols = 28):
    example = np.reshape(img, (num_rows, num_cols))
    plt.matshow(example)
    plt.title("This is digit " + str(label))
    plt.savefig(file_name)

save_img(X_train[0, :], y_train[0], "example.png")

for c in range(num_labels):
    save_img(centroids[c], c, "Centroid_" + str(c) + ".png")


def predict(centroids, X):
    num_centroids = len(centroids)
    distances = []

    for c in range(len(centroids)):
        centroid = centroids[c]
        distance = np.sum((X - centroid) * (X - centroid), axis = 1)
        distances.append(distance)

    labels = np.zeros((X.shape[0]))
    for r in range(X.shape[0]):
        labels[r] = np.argmin(np.array([distances[c][r] for c in range(num_centroids)]))

    return labels

In [ ]:
# Prediction
y_hat = predict(centroids, X_test)

In [ ]:
# Accuracy
acc = np.mean(y_test == y_hat) * 100
print('Accuracy:', acc)

In [ ]:
# Confusion matrix
confusion = np.zeros((num_labels, num_labels))
for i in range(X_test.shape[0]):
    confusion[int(y_test[i]), int(y_hat[i])] += 1
for i in range(num_labels):
    confusion[i, :] = confusion[i, :] / np.sum(confusion[i, :]) * 100
print('Confusion matrix:')
print(confusion)
plt.matshow(confusion)
plt.title("Confusion matrix. Average accuracy = " + str(acc) + ".")
plt.savefig("confusion.png")
print("Done")